In [1]:
import pandas as pd
from gurobipy import *

In [2]:
with open('test_case.txt') as text_file:
    data = text_file.read().splitlines()

data

['Test Sequence with length 20:',
 'CGUCUUCACUACAGCAUCGG',
 '',
 'Test Sequence with length 30:',
 'GACCUUACUGGGUACGAUUUACUGGAGGAC',
 '',
 'Test Sequence with length 50:',
 'GGCCAGACUGGUGGUGUGACUCCAGGCUAACCGGAUACGCGUGCCUCGGG',
 '',
 'Test Sequence with length 100:',
 'UAUGCAGGUCGCGUUUUUCCACUGCCUAGAUAGCUCUGAGGGUACACUUAGUUCAGCACAUAAGAGGGAUCAUACUAGGUCCGCGUCUUACCUCCUACGA',
 '',
 'Test Sequence with length 200:',
 'CCAUGUGGACGAGUUCACGUUGCGGUCUGCCGCGAAGAAUGAGUCGUCCAAUUUUUUUUUCCUGUAUGACGGAUAGAGUGUAGGGUGGGUGAUAGUGUAACUCCGUCCCGGCGUUGUCUAUGGCAGUCUAAUUGUAGUGAGCCAAAAUAUUUCAGAGAGUAACCAUUACAGGCUACCACACGACCCUGAUCCUAUUGUAU',
 '']

In [3]:
s = data[7]
len(s)

50

In [4]:
m = Model("Base_Stacking")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-12


In [5]:
m.setParam('OutputFlag',True)
m.setParam('TimeLimit', 5*60)

Set parameter TimeLimit to value 300


# Pre-Processing Data

In [6]:
ONE_N = list(range(1,len(s)+1))
ONE_N

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50]

## A_i, U_i, C_i, G_i

In [7]:
a = {}
u = {}
c = {}
g = {}

In [8]:
index = 1
for n in s:
    if (n == 'A'):
        a[index] = 1
        u[index] = 0
        c[index] = 0
        g[index] = 0
    if (n == 'U'):
        a[index] = 0
        u[index] = 1
        c[index] = 0
        g[index] = 0
    if (n == 'C'):
        a[index] = 0
        u[index] = 0
        c[index] = 1
        g[index] = 0
    if (n == 'G'):
        a[index] = 0
        u[index] = 0
        c[index] = 0
        g[index] = 1
    
    index += 1

In [9]:
print(a)
print(u)
print(c)
print(g)
print(s)

{1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0}
{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 1, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 1, 43: 0, 44: 0, 45: 0, 46: 1, 47: 0, 48: 0, 49: 0, 50: 0}
{1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 0, 22: 1, 23: 1, 24: 0, 25: 0, 26: 0, 27: 1, 28: 0, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 1, 41: 0, 42: 0, 43: 0, 44: 1, 45: 1, 46: 0, 47

## X_ij

In [10]:
ls_tup = []
for i in range(1,len(s)):
    for j in range(i+1,len(s)+1):
        ls_tup.append((i,j))
        
x = m.addVars(ls_tup,name='X_ij',vtype=GRB.BINARY)
x

{(1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,


## d_ij Minimum Distance Constraint

In [11]:
ls_dtup = []
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        ls_dtup.append((i,j))

d = {}

for i in ONE_N:
    for j in range(i+1,len(s)+1):
        w = min(j-i,len(s)-j+i)
        d[i,j] = w

d

{(1, 2): 1,
 (1, 3): 2,
 (1, 4): 3,
 (1, 5): 4,
 (1, 6): 5,
 (1, 7): 6,
 (1, 8): 7,
 (1, 9): 8,
 (1, 10): 9,
 (1, 11): 10,
 (1, 12): 11,
 (1, 13): 12,
 (1, 14): 13,
 (1, 15): 14,
 (1, 16): 15,
 (1, 17): 16,
 (1, 18): 17,
 (1, 19): 18,
 (1, 20): 19,
 (1, 21): 20,
 (1, 22): 21,
 (1, 23): 22,
 (1, 24): 23,
 (1, 25): 24,
 (1, 26): 25,
 (1, 27): 24,
 (1, 28): 23,
 (1, 29): 22,
 (1, 30): 21,
 (1, 31): 20,
 (1, 32): 19,
 (1, 33): 18,
 (1, 34): 17,
 (1, 35): 16,
 (1, 36): 15,
 (1, 37): 14,
 (1, 38): 13,
 (1, 39): 12,
 (1, 40): 11,
 (1, 41): 10,
 (1, 42): 9,
 (1, 43): 8,
 (1, 44): 7,
 (1, 45): 6,
 (1, 46): 5,
 (1, 47): 4,
 (1, 48): 3,
 (1, 49): 2,
 (1, 50): 1,
 (2, 3): 1,
 (2, 4): 2,
 (2, 5): 3,
 (2, 6): 4,
 (2, 7): 5,
 (2, 8): 6,
 (2, 9): 7,
 (2, 10): 8,
 (2, 11): 9,
 (2, 12): 10,
 (2, 13): 11,
 (2, 14): 12,
 (2, 15): 13,
 (2, 16): 14,
 (2, 17): 15,
 (2, 18): 16,
 (2, 19): 17,
 (2, 20): 18,
 (2, 21): 19,
 (2, 22): 20,
 (2, 23): 21,
 (2, 24): 22,
 (2, 25): 23,
 (2, 26): 24,
 (2, 27): 25,
 (2, 2

In [12]:
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        if d[i,j] < 3:
            x[i,j] = 0

x

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## f_ij

In [13]:
ls_ftup = []
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        ls_ftup.append((i,j))
        


        
f = {}

temp = ''
temp = 'x' + s
temp
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        if (((temp[i] == 'C') & (temp[j] == 'G')) or ((temp[i] == 'G') & (temp[j] == 'C'))):
            f[i,j] = 3
        elif (((temp[i] == 'A') & (temp[j] == 'U')) or ((temp[i] == 'U') & (temp[j] == 'A'))):
            f[i,j] = 2
        elif (((temp[i] == 'G') & (temp[j] == 'U')) or ((temp[i] == 'U') & (temp[j] == 'G'))):
            f[i,j] = 0.1
        elif (((temp[i] == 'A') & (temp[j] == 'C')) or ((temp[i] == 'C') & (temp[j] == 'A'))):
            f[i,j] = 0.05
        else:
            f[i,j] = 0

f

{(1, 2): 0,
 (1, 3): 3,
 (1, 4): 3,
 (1, 5): 0,
 (1, 6): 0,
 (1, 7): 0,
 (1, 8): 3,
 (1, 9): 0.1,
 (1, 10): 0,
 (1, 11): 0,
 (1, 12): 0.1,
 (1, 13): 0,
 (1, 14): 0,
 (1, 15): 0.1,
 (1, 16): 0,
 (1, 17): 0.1,
 (1, 18): 0,
 (1, 19): 0,
 (1, 20): 3,
 (1, 21): 0.1,
 (1, 22): 3,
 (1, 23): 3,
 (1, 24): 0,
 (1, 25): 0,
 (1, 26): 0,
 (1, 27): 3,
 (1, 28): 0.1,
 (1, 29): 0,
 (1, 30): 0,
 (1, 31): 3,
 (1, 32): 3,
 (1, 33): 0,
 (1, 34): 0,
 (1, 35): 0,
 (1, 36): 0.1,
 (1, 37): 0,
 (1, 38): 3,
 (1, 39): 0,
 (1, 40): 3,
 (1, 41): 0,
 (1, 42): 0.1,
 (1, 43): 0,
 (1, 44): 3,
 (1, 45): 3,
 (1, 46): 0.1,
 (1, 47): 3,
 (1, 48): 0,
 (1, 49): 0,
 (1, 50): 0,
 (2, 3): 3,
 (2, 4): 3,
 (2, 5): 0,
 (2, 6): 0,
 (2, 7): 0,
 (2, 8): 3,
 (2, 9): 0.1,
 (2, 10): 0,
 (2, 11): 0,
 (2, 12): 0.1,
 (2, 13): 0,
 (2, 14): 0,
 (2, 15): 0.1,
 (2, 16): 0,
 (2, 17): 0.1,
 (2, 18): 0,
 (2, 19): 0,
 (2, 20): 3,
 (2, 21): 0.1,
 (2, 22): 3,
 (2, 23): 3,
 (2, 24): 0,
 (2, 25): 0,
 (2, 26): 0,
 (2, 27): 3,
 (2, 28): 0.1,
 (2, 29): 

## Q_ij

In [14]:
ls_qtup = []
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        ls_qtup.append((i,j))

q_ij = m.addVars(ls_qtup,name='Q_ij',vtype=GRB.BINARY)
q_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## Q_nj

In [15]:
ls_njtup = []
for j in range(3,len(s)):
    ls_njtup.append((len(s),j))
    
    
q_nj = m.addVars(ls_njtup,name='Q_nj',vtype=GRB.BINARY)
q_nj

{(50, 3): <gurobi.Var *Awaiting Model Update*>,
 (50, 4): <gurobi.Var *Awaiting Model Update*>,
 (50, 5): <gurobi.Var *Awaiting Model Update*>,
 (50, 6): <gurobi.Var *Awaiting Model Update*>,
 (50, 7): <gurobi.Var *Awaiting Model Update*>,
 (50, 8): <gurobi.Var *Awaiting Model Update*>,
 (50, 9): <gurobi.Var *Awaiting Model Update*>,
 (50, 10): <gurobi.Var *Awaiting Model Update*>,
 (50, 11): <gurobi.Var *Awaiting Model Update*>,
 (50, 12): <gurobi.Var *Awaiting Model Update*>,
 (50, 13): <gurobi.Var *Awaiting Model Update*>,
 (50, 14): <gurobi.Var *Awaiting Model Update*>,
 (50, 15): <gurobi.Var *Awaiting Model Update*>,
 (50, 16): <gurobi.Var *Awaiting Model Update*>,
 (50, 17): <gurobi.Var *Awaiting Model Update*>,
 (50, 18): <gurobi.Var *Awaiting Model Update*>,
 (50, 19): <gurobi.Var *Awaiting Model Update*>,
 (50, 20): <gurobi.Var *Awaiting Model Update*>,
 (50, 21): <gurobi.Var *Awaiting Model Update*>,
 (50, 22): <gurobi.Var *Awaiting Model Update*>,
 (50, 23): <gurobi.Var *Awa

## Additional Constraints for Q_ij and Q_nj variables

In [16]:
m.addConstrs(2*q_ij[i,j] <= x[i,j] + x[i+1,j-1] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Model Update*>,
 (1, 23): <gurobi.Constr *Awaiting Mo

In [17]:
m.addConstrs(2*q_nj[len(s),j] <= x[j,len(s)] + x[1,j-1] for j in range(3,len(s)))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Updat

## Q_i,i+1

In [18]:
ls_ii1tup = []
for i in range(1,len(s)):
    ls_ii1tup.append((i,i+1))

m.addVars(ls_ii1tup,name='Q_ij',vtype=GRB.BINARY)
for i in range(1,len(s)):
    q_ij[i,i+1] = 0

q_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## Q_i,i+2

In [19]:
ls_ii2tup = []
for i in range(1,len(s)-1):
    ls_ii2tup.append((i,i+2))

m.addVars(ls_ii2tup,name='Q_ij',vtype=GRB.BINARY)
for i in range(1,len(s)-1):
    q_ij[i,i+2] = 0

q_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## Q_n,n-1

In [20]:
q_nj[len(s),len(s)-1] = 0

## Q_n,n-2

In [21]:
q_nj[len(s),len(s)-2] = 0

## Non-Crossing Constraint

In [22]:
m.addConstrs(quicksum(x[k,l] for k in range(i+1,j) for l in [i for i in list(range(1,len(s)+1)) if i >= j]) <= (10**6)*(1-x[i,j]) for i in range(1,len(s)+1) for j in range(i+1,len(s)+1))

{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Mode

## Character in At most one pair

In [23]:
m.addConstrs(quicksum([x[i,k] for i in range(1,k)]) + quicksum([x[k,j] for j in list(range(k+1,len(s)+1))]) <= 1 for k in ONE_N)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*

# Objective

In [24]:
m.setObjective((quicksum(f[i,j]*x[i,j] for i in ONE_N for j in list(range(i+1,len(s)+1)))) + 
               (quicksum(q_ij[i,j] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1)) + 
               (quicksum(q_nj[len(s),j] for j in range(3,len(s))))),GRB.MAXIMIZE)

In [25]:
# Optimize the model
m.optimize()
# Print the result
status_code = {1: 'LOADED', 2: 'OPTIMAL', 3: 'INFEASIBLE', 4: 'INF_OR_UNBD', 5: 'UNBOUNDED'}
status = m.status
print("The optimization status is {}".format(status_code[status]))
if status == 2:
# Retrieve variables value
    print("Optimal solution:")
    for v in m.getVars():
        if(v.x == 1):
            print("%s = %g" % (v.varName, v.x))
    print("Optimal objective value:\n{}".format(m.objVal))
    print("Runtime is:", m.Runtime)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2450 rows, 2497 columns and 253220 nonzeros
Model fingerprint: 0xfa63f181
Variable types: 0 continuous, 2497 integer (2497 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [5e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 16.5500000
Sparsify removed 203337 nonzeros (85%)
Presolve added 0 rows and 204 columns
Presolve removed 247 rows and 0 columns
Presolve time: 1.99s
Presolved: 2203 rows, 2701 columns, 34971 nonzeros
Variable types: 57 continuous, 2644 integer (2179 binary)

Root relaxation: objective 8.309999e+01, 3350 iterations, 0.31 seconds (0.18 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   

 14770  9418   58.65478  106  822   57.25000   68.17215  19.1%  85.9  290s
 14773  9420   58.60966   36  815   57.25000   68.17215  19.1%  85.9  296s

Cutting planes:
  Gomory: 1
  Cover: 2
  Implied bound: 7
  Clique: 5
  MIR: 14
  Flow cover: 24
  GUB cover: 3
  Inf proof: 1
  Zero half: 17
  RLT: 2
  Relax-and-lift: 7

Explored 14774 nodes (1284430 simplex iterations) in 300.09 seconds (395.25 work units)
Thread count was 4 (of 4 available processors)

Solution count 10: 57.25 55.1 53.1 ... 40.1

Time limit reached
Best objective 5.725000000000e+01, best bound 6.815000000000e+01, gap 19.0393%


KeyError: 9

In [ ]:
m.write("check.lp")